In [2]:
import pandas as pd
import sqlite3

conn = sqlite3.connect('database2.sqlite')

query = "SELECT * FROM sqlite_master"

df_schema = pd.read_sql_query(query, conn)

df_schema.tbl_name.unique()

array(['sqlite_sequence', 'Player_Attributes', 'Player', 'Match',
       'League', 'Country', 'Team', 'Team_Attributes'], dtype=object)

1. Which team scored the most points when playing at home?  

In [31]:
query = """
SELECT * FROM Match
"""

pd.read_sql_query(query, conn).head(5)

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
0,1,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9987,9993,1,...,4.00,1.65,3.40,4.50,1.78,3.25,4.00,1.73,3.40,4.20
1,2,1,1,2008/2009,1,2008-08-16 00:00:00,492474,10000,9994,0,...,3.80,2.00,3.25,3.25,1.85,3.25,3.75,1.91,3.25,3.60
2,3,1,1,2008/2009,1,2008-08-16 00:00:00,492475,9984,8635,0,...,2.50,2.35,3.25,2.65,2.50,3.20,2.50,2.30,3.20,2.75
3,4,1,1,2008/2009,1,2008-08-17 00:00:00,492476,9991,9998,5,...,7.50,1.45,3.75,6.50,1.50,3.75,5.50,1.44,3.75,6.50
4,5,1,1,2008/2009,1,2008-08-16 00:00:00,492477,7947,9985,1,...,1.73,4.50,3.40,1.65,4.50,3.50,1.65,4.75,3.30,1.67


In [34]:
query = """
SELECT home_team_api_id,
SUM(home_team_goal) total_goals_home,
SUM(away_team_goal) total_goals_away
FROM Match 
GROUP BY home_team_api_id, away_team_api_id
ORDER BY total_goals_home DESC
LIMIT 5;
"""

home_team_goals = pd.read_sql_query(query, conn)
home_team_goals

,home_team_api_id,total_goals_home,total_goals_away
0,9925,46,13
1,9925,45,13
2,10179,40,21
3,9823,39,5
4,9931,38,19


2. Did this team also score the most points when playing away?  
* Yes looks like they did

In [35]:
query = """
SELECT away_team_api_id,
SUM(home_team_goal) total_goals_home,
SUM(away_team_goal) total_goals_away
FROM Match 
GROUP BY home_team_api_id, away_team_api_id
ORDER BY total_goals_away DESC
LIMIT 5;
"""

home_team_goals = pd.read_sql_query(query, conn)
home_team_goals

,away_team_api_id,total_goals_home,total_goals_away
0,9925,12,38
1,9931,19,35
2,9925,14,32
3,9925,17,30
4,9931,23,30


3. How many matches resulted in a tie?  

In [45]:
query = """
SELECT home_team_goal, away_team_goal
FROM Match
WHERE home_team_goal = away_team_goal;
"""
print('{} matches resulted in ties'.format(pd.read_sql_query(query, conn).shape[0]))

6596 matches resulted in ties


4. How many players have Smith for their last name? How many have 'smith' anywhere in their name?

In [54]:
query = """
SELECT * FROM Player
WHERE player_name
LIKE '% Smith%';
"""

print('{} players have the last name Smith'.format(pd.read_sql_query(query, conn).shape[0]))

15 players have the last name Smith


In [57]:
query = """
SELECT * FROM Player
WHERE player_name
LIKE '%smith%';
"""

print("{} players have 'smith' somewhere in their name".format(pd.read_sql_query(query, conn).shape[0]))

18 players have 'smith' somewhere in their name


5. What was the median tie score? Use the value determined in the previous question for the number of tie games. *Hint:* PostgreSQL does not have a median function. Instead, think about the steps required to calculate a median and use the [`WITH`](https://www.postgresql.org/docs/8.4/static/queries-with.html) command to store stepwise results as a table and then operate on these results. 

In [85]:
query = """
WITH testing AS (SELECT home_team_goal, away_team_goal
FROM Match
WHERE home_team_goal = away_team_goal
ORDER BY home_team_goal DESC)
SELECT *
FROM testing;
"""
ordered_ties = pd.read_sql_query(query, conn)
ordered_ties

,home_team_goal,away_team_goal
0,6,6
1,5,5
2,5,5
3,4,4
4,4,4
...,...,...
6591,0,0
6592,0,0
6593,0,0
6594,0,0


In [86]:
# i dont understand why this doesnt work...

# query = """
# SELECT
# percentile_cont(0.50) within group (order by home_team_goal asc) as percentile_50
# FROM Match
# """
# pd.read_sql_query(query, conn)

6. What percentage of players prefer their left or right foot? *Hint:* Calculate either the right or left foot, whichever is easier based on how you setup the problem.

In [98]:
query = """
SELECT COUNT(preferred_foot) total_count, preferred_foot FROM Player_Attributes GROUP BY preferred_foot;
"""
pd.read_sql_query(query, conn)

,total_count,preferred_foot
0,0,None
1,44733,left
2,138409,right


In [119]:
# realizing i have to group (i think) by player_api_id first then find the counts.


query = """
WITH temp AS
(SELECT * FROM Player_Attributes GROUP BY player_api_id)
SELECT COUNT(preferred_foot)
player_count, preferred_foot
FROM temp
GROUP BY preferred_foot
;
"""
pd.read_sql_query(query, conn)

,player_count,preferred_foot
0,2687,left
1,8373,right


In [122]:
query = """
WITH temp AS
(SELECT * FROM Player_Attributes GROUP BY player_api_id)
SELECT 100. * COUNT(preferred_foot) / sum(count(preferred_foot)) over ()
percent_of_players, preferred_foot
FROM temp
GROUP BY preferred_foot
;
"""
pd.read_sql_query(query, conn)


,percent_of_players,preferred_foot
0,24.294756,left
1,75.705244,right
